# Installs
___

In [1]:
!pip install rdkit-pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 39.0 MB/s eta 0:00:00


In [2]:
!pip install descriptastorus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 19.7 MB/s eta 0:00:00


In [3]:
!pip install scipy==1.9.0 #probably, you will need to restart the notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 12.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.3
    Uninstalling scipy-1.11.3:
      Successfully uninstalled scipy-1.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


# Imports
___

In [4]:
import pandas as pd
import scipy.stats as stats
import numpy as np
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Draw, Descriptors
from rdkit.Chem.Draw import IPythonConsole
from descriptastorus.descriptors import rdNormalizedDescriptors
from sklearn.decomposition import PCA
import plotly.express as px
import matplotlib.pyplot as plt

# Visualization with PCA method
___

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Загрузим данные из google drive (пропишите свой путь до данных)

In [6]:
PATH_small_data = '/content/drive/MyDrive/atomic_hack_dataset_1400.xlsx' #your path here
PATH_big_data = '/content/drive/MyDrive/atomick_hack_dataset_35000.xlsx' #your path here

In [11]:
data_small = pd.read_excel(PATH_small_data).drop(['Pictures', 'Title'], axis=1)
data_big = pd.read_excel(PATH_big_data , usecols=[2, 3])

Педелаем названия колонок в нижний регистр\
Пробелы заменяем на '_'

In [16]:
data_big = data_big.rename(
    columns={k: k.lower() for k in data_big.columns}
    )

In [15]:
data_small = data_small.rename(columns={
    'IC50, mmg/ml': 'IC50',
    'CC50-MDCK, mmg/ml': 'CC50'
})

rename_dict = {k: k.lower().replace(' ', '_') for k in data_small.columns}

data_small = data_small.rename(columns=rename_dict)

In [17]:
data_big.head()

,ic50,smiles
0,0.000015,[H]\N=C(N)\N[C@@H](C1)[C@@H](NC(=O)C)[C@@H](C=...
1,0.000016,O=C(O)C1=C[C@H](N)[C@@H](NC(=O)C)[C@@H](C1)COC...
2,0.000033,C=CC(=O)N[C@H]([C@H](C1)N)[C@@H](C=C1C([O-])=O...
3,0.000032,CC(=O)N[C@H]([C@H](C1)N)[C@@H](C=C1C([O-])=O)O...
4,0.000041,O=C(O)c1c(O)c(=O)cc([nH]1)-c(c2C)ccc(c2)-c3noc...


In [18]:
data_small.head()

,ic50,cc50,si,molecular_weight,hydrogen_bond_acceptors,hydrogen_bond_donors,polar_sa,smiles
0,2.7,500.0,185.185185,195.307,2,1,32.59,OCC\N=C(\[C@]12C)C[C@@H](C1(C)C)CC2
1,0.7,447.0,638.571429,195.307,2,1,32.59,OCC\N=C(\[C@]12C)C[C@@H](C1(C)C)CC2
2,9.9,144.0,14.545455,250.431,1,0,15.60,CCN(CC)CC\N=C(\[C@@]12C)C[C@H](C1(C)C)CC2
3,8.3,500.0,60.240964,222.377,1,0,15.60,CN(C)CC\N=C(\[C@@]12C)C[C@H](C1(C)C)CC2
4,39.4,143.0,3.629442,239.361,2,0,29.54,CN(C)CC(=O)O[C@H]1C[C@H](CC2)C(C)(C)[C@@]12C


Заполним все пропуски в колонке si

In [19]:
data_small.si = data_small.cc50 / data_small.ic50

In [20]:
data_small.isna().sum()

ic50                       0
cc50                       0
si                         0
molecular_weight           0
hydrogen_bond_acceptors    0
hydrogen_bond_donors       0
polar_sa                   0
smiles                     0
dtype: int64

Функция получения morgan fingerprint из формата smiles

In [24]:
def fp_from_smiles(smiles):

    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048) # generate MF as bit vector
    fp = np.frombuffer(fp.ToBitString().encode(), 'u1') - ord('0') # convert bit vector to 1d numpy array

    return fp

In [21]:
smiles = data_small.smiles

Функция для сжатия признакового пространства до двух признаков с помощью PCA

In [23]:
def pca_df(chem_rep_list):

    pca = PCA(n_components=2)
    pca_arr = pca.fit_transform(chem_rep_list)

    return pd.DataFrame(pca_arr, columns=["Component 1", "Component 2"])

Функция для выведения графика

In [37]:
def graph_chemical_space(
    df: pd.DataFrame,
    smiles_col: str = "SMILES",
    color_col: str = None,
    id_col: str = None,
    chem_rep: str = "rdkit",
    dim_reduction: str = "TSNE",
    graph_title: str = None
):
    """
    df : pandas.DataFrame object
        a pandas dataframe that contains the data plotted in fig.
    smiles_col : str, optional
        name of the column in df containing the smiles plotted in fig (default 'SMILES').
        If provided as a list, will add a slider to choose which column is used for rendering the structures.
    color_col : str, optional
        name of the column in df that will specify the point colors plotted in fig (default None)
    id_col : str, optional
        name of the column in df that will specify id of the points plotted in fig (default None)
    chem_rep : "rdkit" | "mf" | "oc" , optional
        name of the desired chemical representation
    dim_reduction : "PCA" | "TSNE" , optional
        name of desired dimension reduction technique
    graph_title : str, optional
        title of graph
    """

    assert chem_rep in ['rdkit', 'mf', 'oc'], "Please enter a valid chemical representation"
    assert dim_reduction in ["PCA", "TSNE"], "Please enter a valid dimension reduction technique"
    assert smiles_col in df, "The specified smiles_col is not in the dataframe"

    funcs = {
        "mf" : fp_from_smiles,
        "PCA": pca_df
    }

    mol_list = []

    # convert smiles to desired chemical representation
    for smiles in df[smiles_col]:

        mol_list.append(funcs[chem_rep](smiles))

    # apply desired dimension reduction technique
    rd_df = funcs[dim_reduction](mol_list)

    # populate dataframe with color and id columns
    rd_df = rd_df.join(df[smiles_col])

    if id_col is not None:

        rd_df = rd_df.join(df[id_col])

    if color_col is not None:

        rd_df = rd_df.join(df[color_col])

    fig = px.scatter(rd_df, x='Component 1', y='Component 2', color=color_col, title=graph_title)
    return fig

Немного проанализируем данные и уберем слишком большие значения в каждом датасете для лучшего понимания распределения параметров

In [25]:
data_small.describe()

,ic50,cc50,si,molecular_weight,hydrogen_bond_acceptors,hydrogen_bond_donors,polar_sa
count,1468.000000,1468.000000,1.468000e+03,1468.000000,1468.000000,1468.000000,1468.000000
mean,50.277063,186.491530,2.210854e+04,384.739636,3.831744,1.333787,81.273215
std,92.184768,1009.454293,4.771268e+05,153.164268,2.860473,1.803663,54.078778
min,0.000010,0.031928,3.616134e-02,110.157000,0.000000,0.000000,0.000000
25%,2.500000,30.375000,1.833333e+00,283.049250,2.000000,0.000000,39.440000
50%,10.100000,90.400000,5.528073e+00,349.371000,3.000000,1.000000,71.840000
75%,49.450000,300.000000,2.778741e+01,446.508000,5.000000,2.000000,106.160000
max,1000.000000,37646.000000,1.632222e+07,1159.380000,19.000000,11.000000,319.510000


In [26]:
data_big.describe()

,ic50
count,36377.000000
mean,7.813503
std,48.489710
min,0.000010
25%,1.696000
50%,3.635000
75%,5.934000
max,3183.600000


По заданным порогам уберем строки, не удовлетворяющие условиям

In [31]:
data_small_clear_ic50 = data_small[data_small.ic50 < 100]
data_small_clear_si = data_small[data_small.si < 30]
data_small_clear_cc50 = data_small[data_small.cc50 < 250]
data_big_clear_ic50 = data_big[data_big.ic50 < 100]

Посмотрим на оставшееся количество строк

In [35]:
print(
f'''Количество строк после чистки ic50: {len(data_small_clear_ic50)} (prev {len(data_small)}),
Количество строк после чистки cc50: {len(data_small_clear_cc50)} (prev {len(data_small)}),
Количество строк после чистки si50: {len(data_small_clear_si)} (prev {len(data_small)}),
Количество строк после чистки cc50: {len(data_big_clear_ic50)} (prev {len(data_big)})''')

Количество строк после чистки ic50: 1241 (prev 1468), 
Количество строк после чистки cc50: 1010 (prev 1468), 
Количество строк после чистки si50: 1119 (prev 1468),
Количество строк после чистки cc50: 36041 (prev 36377)


In [38]:
graph_chemical_space(data_small_clear_si,
                     smiles_col='smiles',
                     color_col='si',
                     id_col=None,
                     chem_rep='mf',
                     dim_reduction='PCA',
                     graph_title='Morgan Fingerprint + PCA (small_dataset) SI')

In [39]:
graph_chemical_space(data_small_clear_ic50,
                     smiles_col='smiles',
                     color_col='ic50',
                     id_col=None,
                     chem_rep='mf',
                     dim_reduction='PCA',
                     graph_title='Morgan Fingerprint + PCA (small_dataset) IC50')

[07:50:00] Conflicting single bond directions around double bond at index 55.
[07:50:00]   BondStereo set to STEREONONE and single bond directions set to NONE.


In [40]:
graph_chemical_space(data_small_clear_cc50,
                     smiles_col='smiles',
                     color_col='cc50',
                     id_col=None,
                     chem_rep='mf',
                     dim_reduction='PCA',
                     graph_title='Morgan Fingerprint + PCA (small_dataset) CC50')

[07:50:09] Conflicting single bond directions around double bond at index 55.
[07:50:09]   BondStereo set to STEREONONE and single bond directions set to NONE.


In [41]:
graph_chemical_space(data_big_clear_ic50,
                     smiles_col='smiles',
                     color_col='ic50',
                     id_col=None,
                     chem_rep='mf',
                     dim_reduction='PCA',
                     graph_title='Morgan Fingerprint + PCA (big_dataset) IC50')

[07:50:21] Conflicting single bond directions around double bond at index 55.
[07:50:21]   BondStereo set to STEREONONE and single bond directions set to NONE.
[07:50:40] Conflicting single bond directions around double bond at index 7.
[07:50:40]   BondStereo set to STEREONONE and single bond directions set to NONE.
